In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Path to image folders, 
data_path = fr'../Datasets/Test Set'
# os.listdir(data_path)

def remove_white_background(pixels):
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0

    return pixels


def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]



def blueify(pixels):
    return [b for r, g, b in pixels]


def process_files(src):
    data = dict()
    X_red = []
    X_green = []
    X_blue = []
    y = np.empty(shape=[0, 1])
    print("Starting...")
    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24), Image.ANTIALIAS)
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            X_red.append(redify(pixels.copy()))
            X_green.append(greenify(pixels.copy()))
            X_blue.append(blueify(pixels.copy()))
            y = np.append(y, subdir)
    print("Finished")
    return np.asarray(X_red), np.asarray(X_green), np.asarray(X_blue), y

X_red, X_green, X_blue, y = process_files(data_path)
print(X_red)
print(X_green)
print(X_blue)
print(y)


Starting...
Finished
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[[255 255 255 ... 254 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
['Tomato 3' 'Tomato 3' 'Tomato 3']


In [2]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier(max_iter=100)
mlpc.fit(X_red, y)

mlpc.predict(X_green)

# mnist_clf = svm.SVC()
# mnist_clf.fit(x_train, y_train_digit)

array(['Tomato 3', 'Tomato 3', 'Tomato 3'], dtype='<U8')

In [3]:
# plt.imshow(processed["Walnut"]["blue"][3])

In [4]:
big_data_path = fr'../Datasets/train'
X_red_big, X_green_big, X_blue_big, y_big = process_files(big_data_path)

X_train = X_red_big[0:2000]
Y_train = y_big[0:2000]

# validation set
X_val = X_red_big[2000:2500]
Y_val = y_big[2000:2500]

# test set
X_test = X_red_big[2500:3000]
Y_test = y_big[2500:3000]

Starting...
Finished


In [5]:
X_val

array([[255, 255, 255, ..., 254, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 254, 255, ..., 255, 255, 255]])